In [1]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,StorageContext,load_index_from_storage
from llama_index.core import ServiceContext
from langchain_community.llms import Ollama
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.llms.ollama import Ollama

    


In [4]:
documents=SimpleDirectoryReader("pdfs").load_data() # load the documents

KeyboardInterrupt: 

In [7]:
documents

[Document(id_='3d89b2aa-5e4c-4f78-a6d2-8d6b0fc3fdd9', embedding=None, metadata={'page_label': '1', 'file_name': 'file-23.pdf', 'file_path': '/Users/sakshamchaudhary/Documents/Rag/pdfs/file-23.pdf', 'file_type': 'application/pdf', 'file_size': 1012228, 'creation_date': '2024-05-31', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='General Relativity and Gravitation manuscript No.\n(will be inserted by the editor)\nLove– Crelations for elastic hybrid stars\nZoey Zhiyuan Dong1,3·Joshua Cole\nFaggert2,3·Shu Yan Lau3·Kent Yagi3\nReceived: date / Accepted: date\nAbstract Neutron stars (NSs) provide a unique laboratory to study mat-\nter under extreme densities. Recent observations from gravitational and elec-\ntromagnetic wa

In [19]:

system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
# query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")
llm = Ollama(model="llama2", request_timeout=120.0)
llm
# from llama_index.core.prompts.prompts import SimpleInputPrompt
# system_prompt="""
# You are a Q&A assistant. Your goal is to answer questions as
# accurately as possible based on the instructions and context provided.
# """
# ## Default format supportable by LLama2
# query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

# import torch
# from llama_index.llms.huggingface import HuggingFaceLLM
# llm = HuggingFaceLLM(
#     context_window=4096,
#     max_new_tokens=256,
#     generate_kwargs={"temperature": 0.0, "do_sample": False},
#     system_prompt=system_prompt,
#     query_wrapper_prompt=query_wrapper_prompt,
#     tokenizer_name="meta-llama/Meta-Llama-3-8B",
#     model_name="meta-llama/Meta-Llama-3-8B",
#     device_map="auto",
#     # uncomment this if using CUDA to reduce memory usage
#     model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
# )

In [9]:
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/Users/sakshamchaudhary/Documents/Rag/hello/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/var/folders/g2/581g6n5j1tsbywrdg2gxjmlh0000gn/T/ipykernel_50273/2250285859.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [21]:

service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x33529c6b0>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x33529c6b0>, id_func=<function default_id_func at 0x176e87f60>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x327e94230>, callback_manager=<llama_index.core.callba

In [22]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [25]:
index
query_engine=index.as_query_engine()


In [32]:
response=query_engine.query("in Sparse Dictionary Learning, what is the loss function?")
print(response)


The loss function in Sparse Dictionary Learning is the L1-regularized least squares loss function, which is defined as:

L(α, D) = (1/2) \* ||h - Dα||^2 + λ||α||L1

where h is the observed signal, α is the sparse vector, D is the dictionary, and λ is the L1 regularization parameter. The first term in the loss function is the least squares error between the observed signal and the reconstructed signal using the dictionary and the sparse vector. The second term is the L1-norm of the sparse vector, which encourages sparsity in the solution.

In more detail, the loss function can be written as:

L(α, D) = (1/2) \* ||h - Dα||^2 + λ||α||L1
= (1/2) \* ||h - D(αλ)||^2 + λ||α||L1
= (1/2) \* ||h - Dαλ||^2 + λ||α||L1

where αλ is the sparse vector after solving the optimization problem. The first term in the loss function is the least squares error between the observed signal and the reconstructed signal using the dictionary and the sparse vector. The second term is the L1-norm of the sparse vect